In [2]:
import firstStrategy

In [3]:
msg = firstStrategy.ConnectToStream()
endpoint = 'wss://stream.binance.com:9443/ws' 
open_position = False

In [4]:
#firstStrategy.SimulationTrade(msg, endpoint, usd=1000, open_position=False)

In [1]:
import websocket
import json
import pandas as pd

In [6]:
msg = json.dumps({"method": "SUBSCRIBE",
                  "params": ['ethusdt@kline_1m',
                            'ethusdt@kline_15m'],
                                "id":1})
endpoint = 'wss://stream.binance.com:9443/ws' 

In [5]:
endpoint = 'wss://stream.binance.com:9443/ws' 

msg = json.dumps({
"method": "SUBSCRIBE",
"params": ['ethusdt@kline_1m',
           'ethusdt@kline_15m'],
   "id":1})

returns = {'1m': 0, '15m':0}
open_position = False
printing = True
printing2 = True
usd = 1000

def on_open(ws): 
    ws.send(msg)

def on_message(ws, message):
    global df, buy_price, open_position, usd, printing, printing2
    out = json.loads(message)
    df = pd.DataFrame(out['k'],index=[pd.to_datetime(out['E'], unit='ms')])[['s','i','o','c']]
    df['ret ' + df.i.values[0]] = float(df.c)/float(df.o) - 1
    returns[df.i.values[0]] = float(df.c)/float(df.o) - 1
    if printing:
        print('Waiting for buying condition to be fulfilled ...')
        printing = False
    #print('Current price: ' + str(float(df.c)))
    if not open_position and returns['15m'] < 0 and returns['1m'] > 0:
        buy_price = float(df.c)
        open_position = True 
        print('bought for ' + str(buy_price))
    if open_position:
        if printing2:    
            print('target profit: ' + str(round(buy_price * 1.002, 2)))
            print('stop loss: ' + str(round(buy_price * 0.998, 2)))
            print('USD: ' + str(usd))
            print('current price: ' + str(float(df.c)))
            printing2 = False
        if float(df.c) > buy_price * 1.002:
            print('SOLD! profit made: ' + str(round((float(df.c) - buy_price), 2)))
            open_position = False
            printing = True
            printing2 = True
            usd = usd * (((float(df.c) - buy_price) / 100) + 1)
            print('USD: ' + str(usd))
        elif float(df.c) < buy_price * 0.998:
            print('LOSS STOPPED! loss: ' + str(round((float(df.c) - buy_price), 2)))
            open_position = False
            printing = True
            printing2 = True
            usd = usd * (((float(df.c) - buy_price) / 100) + 1)
            print('USD: ' + str(usd))
            print('--------------------')

ws = websocket.WebSocketApp(endpoint, on_message=on_message, on_open=on_open)

ws.run_forever()


Waiting for buying condition to be fulfilled ...
bought for 1607.91
target profit: 1611.13
stop loss: 1604.69
USD: 1000
current price: 1607.91
LOSS STOPPED! loss: -3.63
USD: 963.6999999999989
Waiting for buying condition to be fulfilled ...
bought for 1604.84
target profit: 1608.05
stop loss: 1601.63
USD: 963.6999999999989
current price: 1604.84
SOLD! profit made: 3.26
USD: 995.1166199999989
Waiting for buying condition to be fulfilled ...
bought for 1606.67
target profit: 1609.88
stop loss: 1603.46
USD: 995.1166199999989
current price: 1606.67
LOSS STOPPED! loss: -3.31
USD: 962.1782598779972
Waiting for buying condition to be fulfilled ...
bought for 1603.16
target profit: 1606.37
stop loss: 1599.95
USD: 962.1782598779972
current price: 1603.16
SOLD! profit made: 3.3
USD: 993.9301424539706
Waiting for buying condition to be fulfilled ...
bought for 1605.37
target profit: 1608.58
stop loss: 1602.16
USD: 993.9301424539706
current price: 1605.37
SOLD! profit made: 3.23
USD: 1026.03408605